# Worksheet 6: OOP

## Question 1
### Implement a class Inventory that keeps track of products, their quantities, and their prices.

* `__init__(self):` creates an empty inventory.
* `add(self, name, qty, unit_price)` Adds an item to the inventory.

     * If the item doesn’t exist, create it.

     * If it already exists, increase its quantity and update its price to the new unit_price.
Returns a 2-element tuple (new_quantity, current_unit_price).

* `update_price(self, name, new_price)`
Changes the price of an existing item to new_price.
Returns the new price if the item exists, otherwise returns None.

* `items(self):` returns a new list of tuples (name, quantity, price).
The order is not important, but you must not return a reference to the internal data.
(Changing the returned list must not affect the inventory.)

* `total(self):` computes the total value of the inventory
The contribution of each item to the total cost should be:
i.e. total should add up all of the quantity * price, for each item.

In [80]:
class Inventory():
    def __init__(self):
        self.products = {}
    
    def add(self, name, qty, unit_price):
        if name in self.products:
            self.products[name]["qty"] += qty
            self.products[name]["unit_price"] = unit_price
            return (self.products[name]["qty"], self.products[name]["unit_price"])
        else:
            self.products[name] = {
                "qty": qty,
                "unit_price": unit_price
            }
        
    def update_price(self, name, new_price):
        if name in self.products:
            self.products[name]["unit_price"] = new_price
            return new_price
        else:
            return None

    def items(self):
        items_list = []
        for i in self.products:
            items_list.append((i, self.products[i]["qty"], self.products[i]["unit_price"]))
        return items_list
        
    def total(self):
        total = 0
        for i in self.products:
             total += (self.products[i]["qty"] * self.products[i]["unit_price"])
        return total


## Question 2
### Implement a class `DataStream` representing a finite stream of numbers.

* `__init__(self, items)` where items is any iterable of numbers; store a private list copy.

* `map(self, func):`  new `DataStream` with `func(x)` applied to each element.

* `filter(self, pred):`  new `DataStream` with only items where `pred(x)` is True.

* `reduce(self, combiner, start):` reduce operator using `combiner(acc, x)`, starting at `start`; return the final value.

* `to_list(self):` new list of the items.

Important: `map`/`filter` must not mutate the original stream.

In [89]:
class DataStream():
    def __init__(self, items):
        self.items = items
    
    def map(self, func):
        return DataStream([func(x) for x in self.items])
    
    def filter(self, pred):
        return DataStream([x for x in self.items if pred(x)])
    
    def reduce(self, combiner, start):
        acc = start
        for x in self.items:
            acc = combiner(acc, x)
        return acc

    def to_list(self):
        new_list = []
        for i in self.items:
            new_list.append(i)
        return new_list

## Question 3
### Implement a class Table that represents a simple data table. The table is given as a list of dictionaries, but the first dictionary is special:
it describes the columns (header) and their default values.

Each later dictionary in the list is one row of data.


__init__(self, rows)

The first element in rows is the header: a dictionary of column names with default values.

All later elements are data rows.

Store copies of those rows inside the table.

When a data row is missing a column, insert the header’s default value.

If a data row has extra keys not in the header, ignore them.

Example rows:
>>> rows = [
...    {"city": None, "pop": 0},     # header: column names + default values
...    {"city": "A", "pop": 3},
...    {"city": "B"},                # missing "pop" : __init__ will use default 0
...    {"city": "A", "pop": 2},
...    {"city": "C", "pop": 2, "other" : "a" }    # extra "other" : __init__ will ignore it
... ]

rows(self) : returns a new list of row dictionaries.
Changing the returned list or its dictionaries must not change the table.

group_by(self, key_fn) : returns a new dictionary mapping each key
(computed by key_fn(row)) to a new list of copied rows belonging to that group.
Example:

>>> t = Table(rows)
>>> groups = t.group_by(lambda r: r["city"])
>>> print(groups)
{
    "A": [ {"city": "A", "pop": 3}, {"city": "A", "pop": 2} ],
    "B": [ {"city": "B", "pop": 0} ]
}

aggregate(self, key_fn, agg_fn) : returns a new dictionary mapping each key
to the result of applying agg_fn(list_of_rows_for_that_key).
Example (add up the "pop" field for each different "city":

>>> t.aggregate(lambda r: r["city"], lambda rs: sum(r["pop"] for r in rs)
{"A": 5, "B": 0}

In [106]:
class Table():
    def __init__(self, rows):
        self._header = rows[0].copy()
        self._rows = []
        for i in rows[1:]:
            new_row = {}
            for col in self._header:
                new_row[col] = i.get(col, self._header[col])
            self._rows.append(new_row)
    
    def rows(self):
        rows_copy = []
        for row in self._rows:
            rows_copy.append(row.copy())
        return rows_copy
    
    def group_by(self, key_fn):
        groups = {}
        for row in self._rows:
            key = key_fn(row)
            if key not in groups:
                groups[key] = []
            groups[key].append(row.copy())
        return groups

    def aggregate(self, key_fn, agg_fn):
        groups = self.group_by(key_fn)

        result = {}
        for key, rows in groups.items():
            result[key] = agg_fn(rows)

        return result

## Question 4
### Implement a class that represents a sequence of data, possibly infinite, produced on demand.

Unlike DataStream, it does not store all values. Instead, it calls a producer function every time a new value is needed.

### Rules
* `__init__(self, producer)` takes a zero-argument function producer that, when called:
    * returns the next value, or
    * returns None when there are no more values (for a finite stream).

    Example producer:
    `def make_counter(start=0):
        count = start
        def next_number():
            nonlocal count
            value = count
            count += 1
            return value
        return next_number
example_producer = make_counter(10)`

Every time you call example_producer(), it returns 10, then 11, then 12, and so on.

`next(self):` returns the next value from the producer (or None if finished).

`map(self, func):` returns a new stream object whose producer applies func to each value produced by the current stream.

`filter(self, pred):` returns a new stream object whose producer skips values until pred(value) is True.

`take(self, n):` returns a new list of the first n non-None values, consuming them from the stream.
Stops early if the stream ends.

In [109]:
class Stream:
    def __init__(self, producer):
        self._producer = producer

    def next(self):
        return self._producer()

    def map(self, func):
        def new_producer():
            value = self._producer()
            if value is None:
                return None
            return func(value)

        return Stream(new_producer)

    def filter(self, pred):
        def new_producer():
            while True:
                value = self._producer()
                if value is None:
                    return None
                if pred(value):
                    return value

        return Stream(new_producer)

    def take(self, n):
        result = []
        for _ in range(n):
            value = self._producer()
            if value is None:
                break
            result.append(value)
        return result
